In [40]:
import numpy as np
from scipy import integrate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation

In [41]:
def chua_deriv((x, y, z), t0, alpha=9.5, beta=14.): # definine el sistema de chua
    #se utilizo una generalización que sustituye la función continua lineal a tramos f( x )
    #por una función suave, tal como un  polinomio cubico, en este caso la estudiada por Hirsch y Smale
    f = ((1./16.)*x**3 - (1./6.)*x)
    return [alpha * (y  - f),  x - y + z, -(beta * y)]

In [42]:
N_trajectories = 20

# Elije puntos de inicio al azar, distribuidos uniformemente en torno al punto de equilibrio trivial (0,0,0)
np.random.seed(1)
x0 =  .001 * np.random.random((N_trajectories, 3))

# Resuelve para las trayectorias
t = np.linspace(0, 100, 10000)
x_t = np.asarray([integrate.odeint(chua_deriv, x0i, t)
                  for x0i in x0])

In [43]:
# Establece la figura y eje 3D para la animación 
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')
ax.axis('on')

# Elege un color diferente para cada trayectoria 
colors = plt.cm.jet(np.linspace(0, 1, N_trajectories))

# Establece líneas y puntos 
lines = sum([ax.plot([], [], [], '-', c=c)
             for c in colors], [])
pts = sum([ax.plot([], [], [], 'o', c=c)
           for c in colors], [])

# Prepara el límite de los ejes 
zoom = 3
ax.set_xlim((-zoom, zoom))
ax.set_ylim((-zoom, zoom))
ax.set_zlim((-zoom, zoom))

# Conjunto de puntos de vista: especificado por (grados de altitud, grados  azimutales) 
ax.view_init(30, 0)

# Función de inicialización: trama del fondo de cada trama
def init():
    for line, pt in zip(lines, pts):
        line.set_data([], [])
        line.set_3d_properties([])

        pt.set_data([], [])
        pt.set_3d_properties([])
    return lines + pts

# Función de animación. Esto se llama secuencialmente con el número de cuadro 
def animate(i):
    # un paso por 6 pasos de tiempo por trama. Esto conduce a buenos resultados.
    i = (6 * i) % x_t.shape[1]

    for line, pt, xi in zip(lines, pts, x_t):
        x, y, z = xi[:i].T
        line.set_data(x, y)
        line.set_3d_properties(z)

        pt.set_data(x[-1:], y[-1:])
        pt.set_3d_properties(z[-1:])

    ax.view_init(30, 0.3 * i) # angulo de vista y velocidad de rotacion
    fig.canvas.draw()
    return lines + pts

# inicializa el animador
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=5000, interval=30, blit=True)

# Guardar como MP4. Esto requiere mplayer o ffmpeg instalado. 
# anim.save ( 'lorentz_attractor.mp4', fps = 15, extra_args = ['- vcodec', 'libx264'])

plt.show()